# Model Evaluation


#### Import Libraries

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pulearn import BaggingPuClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, make_scorer, average_precision_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns


#### Define helper functions

In [2]:
# defined performance metrics scorers

def auprc_score(y_true, y_pred):
    return average_precision_score(y_true, y_pred)

def topk(y_true, y_pred, top_k=100, get_mask=False):

    sorted_indices = y_pred.argsort()[::-1]
    top_k_indices = sorted_indices[:top_k]
    y_pred_top_k_mask = np.full(y_true.shape, False, dtype=bool)
    y_pred_top_k_mask[top_k_indices] = True
    top_k_hits = y_true.values[y_pred_top_k_mask].sum()
    
    if get_mask == False:
        return top_k_hits
    else: 
        return y_pred_top_k_mask

# define evaluation function

def evaluate(model, test_X, test_y, scorers):
    # Using a dictionary to store scores associated with each scorer's name
    y_pred = model.predict_proba(test_X)[:,1]
    scores = {name: [] for name, _, _ in scorers}

    for name, scorer, scorer_args in scorers:
        score = scorer(y_true=test_y, y_pred=y_pred, **scorer_args)
        scores[name]=score

    return scores

In [3]:
def evaluate_models(models, models_name, train_X, train_y, test_X, test_y, scorers_with_args, eval_models_path, eval_results_path):

    results = []

    for model in models:
        model.fit(train_X, train_y)

    eval_models_file_name = models_name + '.pickle'
    eval_models_file_path = os.path.join(eval_models_path, eval_models_file_name)

    with open(eval_models_file_path, 'wb') as f:
        pickle.dump(models, f, pickle.HIGHEST_PROTOCOL)

    for model in models:
        result_for_a_model = evaluate(model, test_X, test_y, scorers_with_args)
        print(result_for_a_model)
        results.append(result_for_a_model)

    results_df = pd.DataFrame(results)
    eval_results_file_name = 'eval_result_' + models_name + '.csv'
    eval_results_file_path = os.path.join(eval_results_path, eval_results_file_name)

    results_df.to_csv(eval_results_file_path, index=False)

In [4]:
def get_avg_feature_importances (models):
# Iterate all random forest models in the model list. From each random forest, obtain the feature importance from all its tree. 
# Average the feature importance of each random forest model, and then average the importance of all random forest models
# Finally, plot the average importance of each feature vector

    all_rf_importances = []

    for idx, model in enumerate(models):
        clf = model.named_steps['clf']
        feature_importances = [estimator.feature_importances_ for estimator in clf.estimators_]
        df_feature_importances = pd.DataFrame(feature_importances)
        feature_importances_means = df_feature_importances.mean()
        all_rf_importances.append(feature_importances_means)

    # Convert list of feature importances to a DataFrame
    df_all_rf_importances = pd.concat(all_rf_importances, axis=1).T

    # Calculate the mean feature importance across all models
    average_importance = df_all_rf_importances.mean()

    # Sort by descending importance
    sorted_means = average_importance.sort_values(ascending=False)
    
    return sorted_means

In [5]:
np.set_printoptions(precision=10)

#### Evaluate Candidate PU Learning Models

In [6]:
# evaluation of candidates models by repeating 25 rounds of testing by different random seeds

eva_rounds = 25

np.random.seed(44)
random_states = [np.random.randint(0, 99) for _ in range(eva_rounds)]
print("random_states be used: ", random_states)
    

random_states be used:  [20, 35, 45, 59, 3, 96, 84, 3, 23, 55, 22, 89, 67, 71, 67, 57, 1, 14, 36, 93, 85, 83, 96, 74, 36]


In [7]:
# configure scorers and parameters 
scorers_with_args = [
    ('top_20', topk, {'top_k': 20}),
    ('top_100', topk, {'top_k': 100}),
    ('auprc', auprc_score, {})
]

In [8]:
eval_models_path = os.path.join('eval_models')
eval_results_path = os.path.join('eval_results')

#### Loading the dataset (d=128)

In [9]:
# load datasets

dataset_filename = 'dataset_p_4_q_1_dim_128_walkleng_100_numwalks_500_seed_37.csv'

file_path = os.path.join('data', 'datasets', dataset_filename)
dataset = pd.read_csv(file_path)

#### Splitting the dataset

In [10]:
# X is the data of feature1 to feature128 in the dataset
X = dataset.drop(['id', 'y'], axis=1)
# y is the target value of the last column in the dataset
y = dataset['y']

# split the dataset into training and testing with 80% to 20% proportion and randomly shuffle the data
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size=0.2, random_state=37,stratify=y)

train_indices = train_X.index
test_indices = test_X.index

# print the shape of the data
print(f'Training dataset shape, X: {train_X.shape}')
print(f'Training dataset shape, y: {train_y.shape}')
print(f'Testing dataset shape, X: {test_X.shape}')
print(f'Testing dataset shape, y: {test_y.shape}')

Training dataset shape, X: (12968, 128)
Training dataset shape, y: (12968,)
Testing dataset shape, X: (3242, 128)
Testing dataset shape, y: (3242,)


#### Train and Test on candidates classifiers

For each candidate:

    - create 25 classifiers
    - train each classifier on the training set
    - test each classifier on the testing set
    - evaluate performance by each scorer

 

In [11]:
models_name = 'models_rf_1_' + os.path.splitext(dataset_filename)[0]

models = []

for random_state in random_states:
    base_clf = DecisionTreeClassifier(max_depth=10, max_leaf_nodes=120, min_samples_leaf=2, min_samples_split=2)
    clf = BaggingPuClassifier(n_estimators =200, max_samples= 600, base_estimator=base_clf, n_jobs = -1, random_state= random_state, verbose=0)
    scaler = StandardScaler()
    model = Pipeline([
                    ("scale", scaler),
                    ("clf", clf)
                    ])
    models.append(model)

evaluate_models(models, models_name, train_X, train_y, test_X, test_y, scorers_with_args, eval_models_path, eval_results_path)

c:\Users\Ron\anaconda3\envs\PULearn\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\Ron\anaconda3\envs\PULearn\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\Ron\anaconda3\envs\PULearn\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\Ron\anaconda3\envs\PULearn\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\Ron\anaconda3\envs\PULearn\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.

{'top_20': 4, 'top_100': 23, 'auprc': 0.1467609170061578}
{'top_20': 4, 'top_100': 25, 'auprc': 0.14649288583694797}
{'top_20': 4, 'top_100': 23, 'auprc': 0.1523085946934784}
{'top_20': 5, 'top_100': 22, 'auprc': 0.14439058668724583}
{'top_20': 5, 'top_100': 23, 'auprc': 0.14939435901255493}
{'top_20': 5, 'top_100': 23, 'auprc': 0.1446309576107267}
{'top_20': 3, 'top_100': 22, 'auprc': 0.13369941947132075}
{'top_20': 5, 'top_100': 23, 'auprc': 0.14939435901255493}
{'top_20': 4, 'top_100': 25, 'auprc': 0.14086384928188}
{'top_20': 4, 'top_100': 24, 'auprc': 0.13138542152154328}
{'top_20': 5, 'top_100': 22, 'auprc': 0.14260019525125486}
{'top_20': 5, 'top_100': 23, 'auprc': 0.14360098994253534}
{'top_20': 5, 'top_100': 23, 'auprc': 0.1511268254873056}
{'top_20': 4, 'top_100': 21, 'auprc': 0.15504787468752576}
{'top_20': 5, 'top_100': 23, 'auprc': 0.1511268254873056}
{'top_20': 5, 'top_100': 26, 'auprc': 0.1477469012862166}
{'top_20': 4, 'top_100': 24, 'auprc': 0.1552663959948345}
{'top_2

##### Retrieve samples prediction (top_20) from models for inspection

Inspecting:
- validate the number of predictions, it should be 3242 (total size of testing set)
- validate the number of predictions, it should be 71 (total size of positive in testing set)
- share the prediction to domain expert to check if those predictions are sensible

In [12]:
sample_models_files = 'models_rf_1_dataset_p_4_q_1_dim_128_walkleng_100_numwalks_500_seed_37.pickle'

with open(os.path.join(eval_models_path, sample_models_files), 'rb') as f:
        models_rf_1 = pickle.load(f)

In [13]:
testing_set_samples_model = []
testing_set_samples_model_top_20_mask = []

num_sample = 3

for i in range(num_sample):
    y_pred = models[i].predict_proba(test_X)[:,1]
    testing_set_samples_model.append(y_pred)
    testing_set_samples_model_top_20_mask.append(topk(test_y, y_pred, top_k=20, get_mask=True))
    
id_series = dataset.iloc[test_X.index].id

In [14]:
sample_check_model_test_X_pred_proba = pd.DataFrame()
sample_check_model_test_X_pred_proba['id'] = id_series
sample_check_model_test_X_pred_proba['y'] = test_y
for i in range(num_sample):
    sample_check_model_test_X_pred_proba[f'models_sample_{i}_predict_proba'] = testing_set_samples_model[i]
    sample_check_model_test_X_pred_proba[f'models_sample_{i}_top_20_mask'] = testing_set_samples_model_top_20_mask[i]

In [15]:
sample_check_model_test_X_pred_proba

,id,y,models_sample_0_predict_proba,models_sample_0_top_20_mask,models_sample_1_predict_proba,models_sample_1_top_20_mask,models_sample_2_predict_proba,models_sample_2_top_20_mask
14643,ENSG00000007376,0,0.239069,False,0.260347,False,0.278986,False
10150,ENSG00000169435,0,0.238978,False,0.321041,False,0.305130,False
12511,ENSG00000139697,0,0.224801,False,0.236090,False,0.227781,False
13381,ENSG00000137200,0,0.259852,False,0.217297,False,0.244288,False
4313,ENSG00000143862,0,0.394807,False,0.407500,False,0.453552,False
...,...,...,...,...,...,...,...,...
2771,ENSG00000123570,0,0.235161,False,0.213337,False,0.234353,False
1616,ENSG00000174417,0,0.389891,False,0.339054,False,0.361821,False
7011,ENSG00000134882,1,0.274495,False,0.250580,False,0.206287,False
7009,ENSG00000174917,0,0.254025,False,0.290461,False,0.284509,False


In [16]:
sample_check_model_test_X_pred_proba.to_csv(os.path.join('inspection', 'top20_samples_' + os.path.splitext(sample_models_files)[0] + '.csv'))